# Imports

In [1]:
import pandas as pd
import numpy as np

# Read 2009-2017 Regular Season NFL Play-By-Play Data
"NFL Play by Play 2009-2017 (v4).csv" downloaded from https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016.
Only using regulation plays from scrimmage.

In [2]:
data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
down_data = data.loc[data["down"] > 0] # plays from scrimmage
reg_down_data = down_data.loc[down_data["qtr"] <= 4] # plays from scrimmage in regulation
reg_down_data = reg_down_data.reset_index(drop=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Pre-Processing
# * This section's code is out of date and should be made to run faster. *

Creating "half" and "time_remaining_in_half" columns.

In [3]:
reg_down_data["half"] = np.where(reg_down_data["qtr"] >= 3, 2, 1)
reg_down_data["time_remaining_in_half"] = np.where(reg_down_data["half"] == 2, reg_down_data["TimeSecs"], reg_down_data["TimeSecs"] - 1800)

Creating "length_of_play" column representing time between snaps.

In [4]:
length_of_play = []
for index, row in reg_down_data[:-1].iterrows():
    if reg_down_data.at[index+1,"TimeSecs"] % 900 != 0: # if this was not the play play of the quarter (900 seconds in a quarter)
        length_of_play.append(row["TimeSecs"] - reg_down_data.at[index+1,"TimeSecs"])
    else: # last play of the quarter
        length_of_play.append(row["TimeSecs"] - ((4 - row["qtr"]) * 900))
length_of_play.append(row["TimeSecs"] - (row["qtr"] * 900)) # last play in dataset

reg_down_data = reg_down_data.assign(length_of_play = length_of_play)

### Creating "next_score" and "drive_score" columns

Making list "end_game_indices" containing index in reg_down_data of last play of each game.

In [5]:
game_id_arr = np.array(reg_down_data["GameID"])
game_id_arr_rev = np.fliplr([game_id_arr])[0]
game_id_rev_indices = np.unique(game_id_arr_rev, return_index=True)[1].tolist()
end_game_indices = []
for i in range(len(game_id_rev_indices)):
    end_game_indices.append(len(reg_down_data["GameID"]) - 1 - game_id_rev_indices[i])

Making lists "scoring_plays", "scoring_play_values", and "scoring_play_game_ids" containing index in reg_down_data of each scoring play, the value of said play, and the game_id of said play, respectively.

In [6]:
scoring_plays = [] # list of indices of all scoring plays
scoring_play_values = [] # points scored on scoring play
for index, row in reg_down_data.iterrows():
    if row["sp"] == 1:
        scoring_plays.append(index)
        if row["Touchdown"] == 1:
            point_value = 7 # could be 6, 7 or 8, but will assume 7 for simplicity
        elif row["Safety"] == 1:
            point_value = 2
        else: # field goal
            point_value = 3
        scoring_play_values.append(point_value)
        
    if index % 50000 == 0:
        print("scoring plays: " + str(index) + "/" + str(max(reg_down_data.index)))

scoring_plays_game_ids = [reg_down_data.at[i,"GameID"] for i in scoring_plays] # list of game_ids corresponding to plays in scoring_plays list
print("scoring plays done")

scoring plays: 0/344058
scoring plays: 50000/344058
scoring plays: 100000/344058
scoring plays: 150000/344058
scoring plays: 200000/344058
scoring plays: 250000/344058
scoring plays: 300000/344058
scoring plays done


Creating "next_score" and "drive_score" columns. "next_score" represents the point value of the next scoring play in the game with respect to the current possession team (negative if scored by opponent). "drive_score" represents the point value of the next scoring play if it occured on the same drive as the current play.

In [7]:
next_score = []
drive_score = []
for index, row in reg_down_data.iterrows():
    first_score_in_game = scoring_plays_game_ids.index(row["GameID"])
    last_score_in_game = len(scoring_plays_game_ids) - 1 - scoring_plays_game_ids[::-1].index(row["GameID"])
    scoring_plays_in_game = scoring_plays[first_score_in_game:last_score_in_game+1]
    scoring_plays_remaining_in_game = [i for i in scoring_plays_in_game if reg_down_data.at[i,"TimeSecs"] <= row["TimeSecs"]] # remove previous scoring plays
    scoring_plays_remaining_in_half = [i for i in scoring_plays_remaining_in_game if reg_down_data.at[i,"half"] == row["half"]] # remove scoring plays in subsequent half
    if len(scoring_plays_remaining_in_half) == 0: # no next score
        next_score.append(0)
        drive_score.append(0)
    else:
        next_score_play_index = scoring_plays_remaining_in_half[0] # scoring_plays_remaining_in_half is sorted by time, so this will always be the next scoring play
        scoring_play_values_index = scoring_plays.index(scoring_plays_remaining_in_half[0])
        value = scoring_play_values[scoring_play_values_index]
        if reg_down_data.at[next_score_play_index, "posteam"] == row["posteam"]:
            # check if scoring play was interception, fumble or safety. if so, append negative value, otherwise positive
            if reg_down_data.at[next_score_play_index, "InterceptionThrown"] + reg_down_data.at[next_score_play_index, "Fumble"] + reg_down_data.at[next_score_play_index, "Safety"] > 0:
                next_score.append(-value)
                if reg_down_data.at[next_score_play_index, "Drive"] == row["Drive"]:
                    drive_score.append(-value)
                else:
                    drive_score.append(0)
            else:
                next_score.append(value)
                if reg_down_data.at[next_score_play_index, "Drive"] == row["Drive"]:
                    drive_score.append(value)
                else:
                    drive_score.append(0)
        else:
            if reg_down_data.at[next_score_play_index, "InterceptionThrown"] + reg_down_data.at[next_score_play_index, "Fumble"] + reg_down_data.at[next_score_play_index, "Safety"] > 0:
                next_score.append(value)
                if reg_down_data.at[next_score_play_index, "Drive"] == row["Drive"]:
                    drive_score.append(value)
                else:
                    drive_score.append(0)
            else:
                next_score.append(-value)
                if reg_down_data.at[next_score_play_index, "Drive"] == row["Drive"]:
                    drive_score.append(-value)
                else:
                    drive_score.append(0)

    if index % 10000 == 0:
        print("next_score, drive_score: " + str(index) + "/" + str(max(reg_down_data.index)))
        
reg_down_data = reg_down_data.assign(next_score = next_score)
reg_down_data = reg_down_data.assign(drive_score = drive_score)
print("next_score, drive_score done")

next_score, drive_score: 0/344058
next_score, drive_score: 10000/344058
next_score, drive_score: 20000/344058
next_score, drive_score: 30000/344058
next_score, drive_score: 40000/344058
next_score, drive_score: 50000/344058
next_score, drive_score: 60000/344058
next_score, drive_score: 70000/344058
next_score, drive_score: 80000/344058
next_score, drive_score: 90000/344058
next_score, drive_score: 100000/344058
next_score, drive_score: 110000/344058
next_score, drive_score: 120000/344058
next_score, drive_score: 130000/344058
next_score, drive_score: 140000/344058
next_score, drive_score: 150000/344058
next_score, drive_score: 160000/344058
next_score, drive_score: 170000/344058
next_score, drive_score: 180000/344058
next_score, drive_score: 190000/344058
next_score, drive_score: 200000/344058
next_score, drive_score: 210000/344058
next_score, drive_score: 220000/344058
next_score, drive_score: 230000/344058
next_score, drive_score: 240000/344058
next_score, drive_score: 250000/344058


### Creating "winner" and "is_winner" columns

Creating "winner" column which indicates who went on to win the game; "tie" if game reached overtime.

In [8]:
unique_game_ids = [reg_down_data.at[i,"GameID"] for i in end_game_indices]
final_score_diff = [reg_down_data.at[i,"ScoreDiff"] for i in end_game_indices]
last_posteam_of_game = [reg_down_data.at[i,"posteam"] for i in end_game_indices]
last_opp_of_game = [reg_down_data.at[i,"DefensiveTeam"] for i in end_game_indices]

winning_teams = []
for i in range(len(final_score_diff)):
    if final_score_diff[i] > 0:
        winning_teams.append([unique_game_ids[i], last_posteam_of_game[i]])
    elif final_score_diff[i] < 0:
        winning_teams.append([unique_game_ids[i], last_opp_of_game[i]])
    else:
        winning_teams.append([unique_game_ids[i], "tie"]) # all OT games fall into this (dataset is just regulation)

game_id_list = reg_down_data["GameID"].tolist()
winner = [""] * len(game_id_list)
for i in range(len(winning_teams)):
    current_game_id = winning_teams[i][0]
    first_play = game_id_list.index(current_game_id)
    last_play = len(game_id_list) - 1 - game_id_list[::-1].index(current_game_id)
    for j in range(first_play,last_play+1):
        winner[j] = winning_teams[i][1]
reg_down_data = reg_down_data.assign(winner = winner)

Creating "is_winner" column representing whether possession team went on to win game; 1 if posteam won, 0.5 if game reached overtime,  otherwise.

In [9]:
reg_down_data["is_winner"] = np.where(reg_down_data["winner"] == reg_down_data["posteam"], 1, np.where(reg_down_data["winner"] == "tie", 0.5, 0))

Creating "pos_timeouts", "def_timeouts", and "total_score" columns, as well as additional dummy columns "is_home_team" and "is_x_down" for x in first through fourth.

In [10]:
reg_down_data["pos_timeouts"] = np.where(reg_down_data["posteam"] == reg_down_data["HomeTeam"], reg_down_data["HomeTimeouts_Remaining_Pre"], reg_down_data["AwayTimeouts_Remaining_Pre"])
reg_down_data["def_timeouts"] = np.where(reg_down_data["posteam"] == reg_down_data["HomeTeam"], reg_down_data["AwayTimeouts_Remaining_Pre"], reg_down_data["HomeTimeouts_Remaining_Pre"])

In [11]:
reg_down_data["total_score"] = reg_down_data["PosTeamScore"] + reg_down_data["DefTeamScore"]

In [12]:
reg_down_data["is_home_team"] = np.where(reg_down_data["posteam"] == reg_down_data["HomeTeam"], 1, 0)

In [13]:
reg_down_data["is_first_down"] = np.where(reg_down_data["down"] == 1, 1, 0)
reg_down_data["is_second_down"] = np.where(reg_down_data["down"] == 2, 1, 0)
reg_down_data["is_third_down"] = np.where(reg_down_data["down"] == 3, 1, 0)
reg_down_data["is_fourth_down"] = np.where(reg_down_data["down"] == 4, 1, 0)

# Save Data

In [14]:
reg_down_data.to_csv("reg_down_data.csv", encoding="utf-8", index=False)